7-8 케라스(Keras) 훑어보기

1. 전처리(Preprocessing)

Tokenizer() : 토큰화와 정수 인코딩을 위해 사용된다. 훈련 데이터로부터 단어 집합을 생성하고, 해당 단어 집합으로 임의의 문장을 정수 인코딩하는 과정을 보여준다.

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
train_text = "The earth is an awesome place live"

# 단어 집합 생성
tokenizer.fit_on_texts([train_text])

# 정수 인코딩
sub_text = "The earth is an great place live"
sequences = tokenizer.texts_to_sequences([sub_text])[0]

print("정수 인코딩 : ",sequences)
print("단어 집합 : ",tokenizer.word_index )

정수 인코딩 :  [1, 2, 3, 4, 6, 7]
단어 집합 :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


pad_sequence() : 전체 훈련 데이터에서 각 샘플의 길이는 서로 다를 수 있는데, 보통 숫자 0을 넣고 길이가 다른 샘플들의 길이를 맞춰준다

In [6]:
pad_sequences([[1,2,3],[3,4,5,6],[7,8]], maxlen = 3, padding = 'pre') # padding = 'post' 로 하면 0이 앞에서부터 채운다.

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]])

2. 워드 임베딩(Word Embedding)
* 워드 임베딩이란 텍스트 내의 단어들을 밀집 벡터(dense vector)로 만드는 것을 말한다.

Embedding() : 단어를 밀집 벡터로 만드는 역할을 한다. 인공 신경망 용어로는 임베딩 층을 만드는 역할을 한다.

In [ ]:
# 1. 토큰화
tokenized_text = [['Hope','to','see','you','soon'],['Nice','to','see','you','again']]

# 2. 각 단어에 대한 정수 인코딩
encoded_text = [[0,1,2,3,4],[5,1,2,3,6]]

# 3. 위 정수 인코딩 데이터가 아래의 임베딩 층의 입력이 된다.
vocab_size = 7
embedding_dim = 2
Embedding(vocab_size, embedding_dim, input_length = 5)

vocab_size = 단어 집합의 크기, 총 단어의 개수\
embedding_dim = 임베딩 벡터의 출력 차원, 결과로서 나오는 임베딩 벡터의 크기\
input_length = 입력 시퀀스의 길이

3. 모델링(Modeling)
* Sequential() : 인공 신경망 챕터에서 배운 입력층, 은닉층, 출력층을 구성하기 위해서 사용한다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(...) # 층 추가
model.add(...) # 층 추가
model.add(...) # 층 추가

Embedding()을 통해 생성하는 임베딩 층(embedding layer)을 추가하는 예시

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim, input_length))

전결합층(fully-connected layer)을 추가하는 예시

In [ ]:
model = Sequential()
model.add(Dense(1, input_dim = 3, activation = 'relu'))

4. 컴파일(Compile)과 훈련(Training)

compile() : 모델을 기계가 이해할 수 있도록 컴파일한다. 손실 함수와 옵티마이저, 메트릭 함수를 선택한다.

In [7]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

vocab_size = 10000
embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])

fit() : 모델을 학습한다. 즉, 모델의 훈련을 시작하는것을 의미

In [ ]:
# 위의 compile() 코드의 연장선상인 코드
model.fit(X_train, y_train, epochs = 10, batch_size = 32)

In [ ]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32, verbose = 0, validation_data(X_val, y_val))

validation_data(X_val, y_val) = 검증데이터를 사용한다. 검증데이터의 오차(loss)가 낮아지다가 높아지기 시작하면 이는 과적합의 신호이다.

validation_split = validation_data와 동일하게 검증 데이터를 사용하기 위한 용도로 validation_data 대신 사용가능하다. 검증 데이터를 지정하는게 아닌 훈련 데이터와 레이블인 X_train, y_train에서 일정 비율 분리하여 사용한다.

In [ ]:
# 훈련 데이터의 20%를 검증 데이터로 사용
model.fit(X_train, y_train, epochs = 10, batch_size = 32, verbose = 0, validation_split = 0.2)

verbose = 학습 중 출력되는 문구를 설정한다.
- 0 : 아무 것도 출력하지 않는다. 
- 1 : 훈련의 진행도를 보여주는 진행 막대를 보여준다. 
- 2 : 미니 배치마다 손실 정보를 출력한다

5. 평가(Evaluation)와 예측(Prediction)

evaluate() : 테스트 데이터를 통해 학습한 모델에 대한 정확도를 평가한다.

In [ ]:
# 위의 fit() 코드의 연장선상인 코드
model.evaluate(X_test, y_test, batch_size = 32)

predict() : 임의의 입력에 대한 모델의 출력값을 확인한다.

In [ ]:
# 위의 fit() 코드의 연장선상인 코드
model.predict(X_input, batch_size = 32)

6. 모델의 저장(Save)과 로드(Load)

save() : 인공 신경망 모델을 hdf5 파일에 저장한다.

In [ ]:
model.save("model_name.h5")

load_model() : 저장해둔 모델을 불러온다.

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("model_name.h5")